In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import itertools
from sklearn import metrics
%matplotlib inline
import datetime

In [2]:
#path = "../../../Google Drive/Data_science/NYU/Machine Learning/ML Project (Collisions)/" #Joe
path = "../../../../Google Drive/ML Project (Collisions)/" # Joyce
# path = "" # Lucas

In [ ]:
with open(path+"collisions_1hot.pkl", 'rb') as infile:
    df = pickle.load(infile)

In [ ]:
df = df.sort_values(by=['date_time'])

In [ ]:
for column in ['date_time', 'rise_time', 'set_time']:
    df[column] = pd.to_datetime(df[column])
    df[column] = df[column].dt.hour*60 + df[column].dt.minute
    
    if np.sum(df[column].isnull()) > 0:
        df[column+'_nan'] = df[column].isnull()
        df[column].fillna(df[column].mean(), inplace=True)

In [ ]:
df = df.drop(['injured', 'killed'], axis=1)

In [ ]:
train_size = int(np.floor(0.7*df.shape[0]))

In [ ]:
small = df[df.date_time.dt.date > datetime.date(2015, 9, 12)]

In [ ]:
valsize = int(np.floor(2/3*small.shape[0]))

In [ ]:
valsize

In [ ]:
small.iloc[:valsize]

In [ ]:
X_train = df.drop(['injured_or_killed'], axis=1).iloc[:train_size]
y_train = df['injured_or_killed'].iloc[:train_size]
X_test = df.drop(['injured_or_killed'], axis=1).iloc[train_size:]
y_test = df['injured_or_killed'][train_size:]

In [ ]:
rf = RandomForestClassifier(n_estimators=50, max_depth=20, min_samples_leaf=10)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
100 - np.sum(y_test)/len(y_test) * 100

In [ ]:
rf.score(X_test, y_test)*100

In [ ]:
np.sum(predictions)

In [ ]:
predictions = rf.predict(X_train)
auc = metrics.roc_auc_score(y_train, predictions)
sensitivity = y_train.astype(int).dot(predictions.astype(int))/np.sum(y_train)*100
specificity = (y_train == 0).astype(int).dot((predictions==0).astype(int))/np.sum((y_train == 0))*100


print ("Train AUC: %s, Sensitivity: %s, Specificity: %s, Score: %s" % (auc, sensitivity,\
                                                                      specificity,\
                                                                      (10*sensitivity+specificity)/11))

In [ ]:
predictions = rf.predict(X_test)
auc = metrics.roc_auc_score(y_test, predictions)
sensitivity = y_test.astype(int).dot(predictions.astype(int))/np.sum(y_test)*100
specificity = (y_test == 0).astype(int).dot((predictions==0).astype(int))/np.sum((y_test == 0))*100


print ("Test AUC: %s, Sensitivity: %s, Specificity: %s, Score: %s" % (auc, sensitivity,\
                                                                      specificity,\
                                                                      (10*sensitivity+specificity)/11))


In [ ]:
fig = plt.figure(figsize=(14, 10))

num_features = 30

y_pos = np.arange(num_features)
importances = np.vstack((rf.feature_importances_, X_train.columns.values)).T
topX = importances[importances[:,0].argsort()][::-1][:num_features]

plt.bar(y_pos, topX[:, 0])
plt.xticks(y_pos, topX[:, 1])
fig.autofmt_xdate()

## Using H2O without 1 hot encoding

In [31]:
with open(path+"collisions_no1hot.pkl", 'rb') as infile:
    df_no1hot = pickle.load(infile)

In [32]:
df_no1hot['year'] = pd.to_datetime(df_no1hot['date_time']).dt.year

In [39]:
df_no1hot = df_no1hot.sort_values('date_time')

In [47]:
view_date = pd.to_datetime(df_no1hot['date_time'])

train_indices = (0, np.sum(view_date < datetime.date(2015,9,12))-1)
val_indices = (train_indices[1]+1,\
               train_indices[1] + \
               np.sum((view_date >= datetime.date(2015,9,12)) & (view_date < datetime.date(2016,7,31))))
test_indices = (val_indices[1]+1,\
               val_indices[1] + np.sum(view_date >= datetime.date(2016,7,31)))

In [54]:
df_no1hot.to_csv(path+"no1hot.csv")

In [5]:
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [6]:
h2o.init(ip="127.0.0.1",max_mem_size_GB = 4)

Checking whether there is an H2O instance running at http://127.0.0.1:54321. connected.


H2O cluster uptime:,4 days 2 hours 31 mins
H2O cluster version:,3.10.4.3
H2O cluster version age:,15 days
H2O cluster name:,H2O_from_python_joycewu_aayuou
H2O cluster total nodes:,1
H2O cluster free memory:,3.099 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [7]:
h2o.connect()

Connecting to H2O server at http://localhost:54321... successful.


H2O cluster uptime:,4 days 2 hours 31 mins
H2O cluster version:,3.10.4.3
H2O cluster version age:,15 days
H2O cluster name:,H2O_from_python_joycewu_aayuou
H2O cluster total nodes:,1
H2O cluster free memory:,3.099 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


<H2OConnection to http://localhost:54321, session _sid_b87e>

In [77]:
dataFrame = h2o.upload_file(path=path+"no1hot.csv", destination_frame="no1hot")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [78]:
dataFrame = dataFrame.drop(['vehicle_type_code_1_nan',
 'vehicle_type_code_2_nan',
 'vehicle_type_code_3_nan',
 'vehicle_type_code_4_nan',
 'vehicle_type_code_5_nan',
 'borough_nan', 'injured', 'killed', 'C1'])

In [79]:
train_df = dataFrame[train_indices[0]:train_indices[1], :]
val_df = dataFrame[val_indices[0]:val_indices[1], :]
test_df = dataFrame[test_indices[0]:test_indices[1], :]

In [63]:
def score_confusion(confusion):
    sensitivity = confusion[1][1]/float(np.sum(confusion[1]))
    specificity = confusion[0][0]/float(np.sum(confusion[0]))
    return sensitivity, specificity

In [80]:
grid_search = []

In [81]:
n_trees = [50, 100, 200]
max_depth = [10, 20, 50, 100]
min_rows = [None, 10, 50, 100]

for i in itertools.product(n_trees,max_depth,min_rows):
    
    if i not in [i[0] for i in grid_search]:
    
        rf = H2ORandomForestEstimator(
        model_id = "rf",
        ntrees=i[0],
        max_depth = i[1],
        min_rows = i[2],
        stopping_rounds=2,
        seed=1000000)

        rf.train(train_df.drop('injured_or_killed').columns, 'injured_or_killed',\
                 training_frame=train_df, validation_frame=val_df)

        results = [i,\
                    rf.auc(train=True),\
                    rf.auc(valid=True),\
                    score_confusion(rf.confusion_matrix(train=True).to_list()),\
                    score_confusion(rf.confusion_matrix(valid=True).to_list())]
        print ("%s finished!" % (str(i)))
        print ("Train AUC: %s, Valid AUC: %s, Train S&S: %s, Valid S&S: %s" % (results[1], results[2], results[3], results[4]))
        
        grid_search.append(results)   
        

drf Model Build progress: |███████████████████████████████████████████████| 100%
(50, 10, None) finished!
Train AUC: 0.7964105178756552, Valid AUC: 0.7660665893018703, Train S&S: (0.5537270544643363, 0.896978568997239), Valid S&S: (0.556944010552258, 0.8502693579339206)
drf Model Build progress: |███████████████████████████████████████████████| 100%
(50, 10, 10) finished!
Train AUC: 0.7970170234669708, Valid AUC: 0.766704565876601, Train S&S: (0.555352855549464, 0.8958951164544627), Valid S&S: (0.5517720156895415, 0.8536681232803865)
drf Model Build progress: |███████████████████████████████████████████████| 100%
(50, 10, 50) finished!
Train AUC: 0.7964908410998762, Valid AUC: 0.76384546954093, Train S&S: (0.5513167098323156, 0.8982244281840045), Valid S&S: (0.5580547745496199, 0.8490747805445732)
drf Model Build progress: |███████████████████████████████████████████████| 100%
(50, 10, 100) finished!
Train AUC: 0.7960537976866242, Valid AUC: 0.7638900979738261, Train S&S: (0.5524509896

In [ ]:
grid_search

In [ ]:
fig = plt.figure(figsize=(14, 10))

num_features = 30

y_pos = np.arange(num_features)

plt.bar(y_pos, rf.varimp(True)['percentage'][:num_features])
plt.xticks(y_pos, rf.varimp(True)['variable'][:num_features])
fig.autofmt_xdate()